In [1]:
%load_ext autoreload
%autoreload 2

from Pytorch_VAE_LSTM import VAE
from dataset.modules.music_dataset import TorchMusicDataset

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import matplotlib.pyplot as plt
import numpy as np

dataset = TorchMusicDataset("./dataset/cleaned_data/cleaned_data")

In [2]:
songs = []
for i in range(8):
    obj = dataset.__getitem__(i)
    songs.append(obj['time_series'].reshape(obj['time_series'].numpy().shape[0], 1))

In [3]:
batch_size = 8
n_in = songs[0].shape[0]
songs = np.array(songs)
songs = torch.tensor(songs, dtype=torch.float32)

dataset = TensorDataset(songs, songs) 
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

vae = VAE(timesteps=n_in, features=1, latent_dim=4, device=device).to(device)
vae.optimizer = optim.Adam(vae.parameters(), lr=1e-3)

# Training loop
epochs = 300
for epoch in range(epochs):
    for batch_x, _ in dataloader:
        batch_x = batch_x.to(device)
        loss = vae.training_step(batch_x)
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss}')
    
num_samples = 1
generated_sequences = vae.generate_sequences(num_samples)

RuntimeError: cuDNN error: CUDNN_STATUS_NOT_SUPPORTED. This error may appear if you passed in a non-contiguous input.